In [1]:
!nvidia-smi

Tue May 21 23:38:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      On  | 00000000:00:03.0 Off |                    0 |
| N/A   58C    P8              18W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/jupyter/unattended-baggage-yolov8-modification


In [3]:
!pip install opencv-python
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 8.5 MB/s eta 0:00:00


In [1]:
!pip install -q \
clearml

In [13]:
!pip install -e .

Obtaining file:///home/jupyter/unattended-baggage-yolov8-modification
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.2.2-0.editable-py3-none-any.whl size=16601 sha256=01a714cb67710d0bd4f7bed197a18bfae3defe2430d545d512f614a3413993b3
  Stored in directory: /var/tmp/pip-ephem-wheel-cache-0uxqm91l/wheels/16/24/4e/77eb3faafdc0df8f4050b5c10cf91dc4af4801edeb0d66c251
Successfully built ultralytics
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.2.2
    Uninstalling ultralytics-8.2.2:
      Successfully uninstalled ultralytics-8.2.2


## Setup ClearML

In [3]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=3YPE676I4P8W2UIQKUCO
%env CLEARML_API_SECRET_KEY=dn1W9hkP09NFWXsATQENuBhUdol1ZrOVGFioO8ucf3kKKZJ2Q4

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=3YPE676I4P8W2UIQKUCO
env: CLEARML_API_SECRET_KEY=dn1W9hkP09NFWXsATQENuBhUdol1ZrOVGFioO8ucf3kKKZJ2Q4


## YOLOv8 baseline

In [10]:
task.close()

In [11]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="baseline 22 May 2024 Model Training YOLOv8",
)

ClearML Task: created new task id=c19277e50b5e40919241724e4f6e8866
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/c19277e50b5e40919241724e4f6e8866/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [14]:
# Step 2: Selecting the YOLOv8 Model, for this case baseline, yolov8n
model_variant = "ultralytics/cfg/models/v8/yolov8.yaml"
task.set_parameter("model_variant", model_variant)

In [16]:
# Step 3: Loading the YOLOv8 Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Transferred 355/355 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 configuration used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 16  # Smaller batch size 
device = 0,1,2,3,4,5,6,7 # GPU device
epochs = 100 # Train for 100 epochs
workers = 8 # workers for GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 30 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for baseline training

In [17]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=-1,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 16,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'baseline_22_May_unattended_baggage_detection_yolov8',
                      save = True,
                      plots = True,
                      patience = 30,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': -1,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 16,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'baseline_22_May_unattended_baggage_detection_yolov8',
 'save': True,
 'plots': True,
 'patience': 30,
 'verbose': True}

In [18]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8.yaml, data=/home/jupyter/gcs/unattended-bagga

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline_22_May_unattended_baggage_detection_yolov8/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 96 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline_22_May_unattended_baggage_detection_yolov8
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

Exception in thread Thread-2 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/envs/pytorch/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-2 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/envs/pytorch/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/pytorch/lib/python3.10/threading.py", line 953, in run
    self.run()
  File "/opt/conda/envs/pytorch/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 49, in _pin_memory_loop
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 49, in _pin_memory_loop
    do_one_step()
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin

YOLOv8 summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:03<00:00, 11.25it/s]


                   all        145        586        0.9      0.703       0.84      0.552
              backpack        145        192      0.929      0.751      0.862      0.521
               handbag        145        310      0.893      0.668      0.845      0.475
              suitcase        145         84      0.879       0.69      0.813      0.662
Speed: 0.6ms preprocess, 13.6ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/baseline_22_May_unattended_baggage_detection_yolov8
2024-05-22 03:48:09,040 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/baseline%2022%20May%202024%20Model%20Training%20YOLOv8.c19277e50b5e40919241724e4f6e8866/models/best.pt


Connecting multiple input models with the same name: `best`. This might result in the wrong model being used when executing remotely


## YOLOv8 +AIFI backbone

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="AIFI 21 May 2024 Model Training",
)


ClearML Task: created new task id=fb6db6811c634d1b969286debae31bc3
2024-05-21 07:02:09,894 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/fb6db6811c634d1b969286debae31bc3/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [5]:
# Step 2: Selecting the YOLOv8 Model, for this case AIFI
model_variant = "ultralytics/cfg/models/v8/yolov8-aifi.yaml"
task.set_parameter("model_variant", model_variant)

In [6]:
# Step 3: Loading the YOLOv8 AIFI Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-21 07:02:59,653 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 307/355 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 AIFI Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size for Tesla T4
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_aifi #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [7]:

# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=-1,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=250,
                      workers = 16,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'AIFI_new_unattended_baggage_detection',
                      save = True,
                      plots = True,
                      patience = 30,
                      verbose = True)
task.connect(args)


{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': -1,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 250,
 'workers': 16,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'AIFI_new_unattended_baggage_detection',
 'save': True,
 'plots': True,
 'patience': 30,
 'verbose': True}

In [10]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-aifi.yaml, data=/home/jupyter/gcs/unattended-

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_new_unattended_baggage_detection3/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 66 weight(decay=0.0005), 67 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 96 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_new_unattended_baggage_detection3
Starting training for 250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.886      0.794      0.864      0.597

250 epochs completed in 2.060 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_new_unattended_baggage_detection3/weights/last.pt, 7.6MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_new_unattended_baggage_detection3/weights/best.pt, 7.6MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_new_unattended_baggage_detection3/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22

Exception in thread Thread-2 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/envs/pytorch/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/pytorch/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 49, in _pin_memory_loop
    do_one_step()
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 26, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/envs/pytorch/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/multiprocessing/reductions.py", line 305, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/envs/pytorch/lib/python3.10/multiprocessing/resource_s

YOLOv8-aifi summary (fused): 173 layers, 3631769 parameters, 0 gradients, 8.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:02<00:00, 13.79it/s]


                   all        145        586      0.886      0.793      0.865      0.597
              backpack        145        192      0.896      0.804      0.861      0.554
               handbag        145        310      0.813      0.802      0.875      0.503
              suitcase        145         84       0.95      0.774      0.858      0.733
Speed: 0.6ms preprocess, 14.3ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/AIFI_new_unattended_baggage_detection3
2024-05-21 10:00:50,724 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/AIFI%2021%20May%202024%20Model%20Training.fb6db6811c634d1b969286debae31bc3/models/best.pt


## YOLOv8 + BilevelRoutingAttention

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="3 May 2024 Model Training BiLevelRouting Attention",
)


ClearML Task: created new task id=616edbb8c0484e29a606b2d2c38ea22a
2024-05-03 08:11:28,572 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/616edbb8c0484e29a606b2d2c38ea22a/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [5]:
# Step 2: Selecting the YOLOv8 Model, for this case BilevelRoutingAttention
model_variant = "ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml"
task.set_parameter("model_variant", model_variant)

In [6]:
# Step 3: Loading the YOLOv8 BilevelRoutingAttention Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-03 08:12:03,967 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 186/379 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 BiLevelRouting Attention Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 8  # Smaller batch size for Tesla T4
device = 0,1,2,3,4,5,6,7 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_bilevelroutingattention #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [8]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_bilevelroutingattention_multihead',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_bilevelroutingattention_multihead',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [9]:
# Step 5: Initiating Model Training
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-BiLevelRoutingAttention.yaml, data=/home/jupyt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.26.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 76 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.789      0.668      0.708      0.446
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     91/100      8.49G       1.05     0.8027       1.23          1       1280: 100%|██████████| 145/145 [00:36<00:00,  3.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 15.41it/s]


                   all        145        586      0.744      0.662      0.681      0.406
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 66, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

91 epochs completed in 1.042 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/last.pt, 7.2MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/best.pt, 7.2MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 16.85it/s]


                   all        145        586      0.855      0.698      0.802      0.495
              backpack        145        192      0.802      0.719       0.79      0.457
               handbag        145        310      0.866      0.635       0.78      0.388
              suitcase        145         84      0.897      0.738      0.836      0.641
Speed: 0.4ms preprocess, 25.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_bilevelroutingattention_multihead
2024-05-03 09:17:18,404 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/3%20May%202024%20Model%20Training%20BiLevelRouting%20Attention.616edbb8c0484e29a606b2d2c38ea22a/models/best.pt


### YOLOv8 + Shuffle Attention

In [11]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="Shuffle Attention 3 May 2024 Model Training Multi Head",
)


ClearML Task: created new task id=ef6340197a6e45ceaf83fe6ffab81685
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/ef6340197a6e45ceaf83fe6ffab81685/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [12]:
# Step 2: Selecting the YOLOv8 Model, for this case ShuffleAttention
model_variant = "ultralytics/cfg/models/v8/yolov8-ShuffleAttention.yaml"
task.set_parameter("model_variant", model_variant)

In [13]:
# Step 3: Loading the YOLOv8 ShuffleAttention Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.


Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


Transferred 186/379 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

## YOLOv8 Shuffle Attention Configuration Used
```
model = yolov8n.pt
data = /home/jupyter/gcs/unattended-baggage/unattended_baggage_march_2024_experiment/data.yaml # Dataset config file
imgsz = 1280 # Image size
batch = 4  # Smaller batch size for Tesla T4
device = 0,1 # GPU device
epochs = 100 # Train for 100 epochs
workers = 2 # Lower workers for smaller GPU
optimizer = SGD # Default YOLO SGD
save_period = 5 # Save every 5 epochs
name = unattended_baggage_detection_shuffleattention #experiment name
save = True #save train checkpoints and predict results
plots = True #save plots and images during train/val
patience = 25 #epochs to wait for no observable improvement for early stopping of training
verbose = True #whether to print verbose output
```
Rest use defaults for training

In [14]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=8,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 2,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_shuffleattention_multihead',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 8,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 2,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_shuffleattention_multihead',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [15]:
# Step 5: Initiating Model Training for ShuffleAttention
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-ShuffleAttention.yaml, data=/home/jupyter/gcs/

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.26.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 61 weight(decay=0.0), 72 weight(decay=0.0005), 75 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 16 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     94/100     0.799G     0.9945     0.7234      1.182          1       1280: 100%|██████████| 145/145 [00:32<00:00,  4.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 15.02it/s]


                   all        145        586       0.84      0.672      0.777      0.485
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 69, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

94 epochs completed in 0.968 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/last.pt, 6.3MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/best.pt, 6.3MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                    

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:04<00:00, 18.11it/s]


                   all        145        586      0.816      0.716      0.786      0.487
              backpack        145        192      0.856      0.713      0.769      0.437
               handbag        145        310      0.708        0.7      0.776      0.399
              suitcase        145         84      0.885      0.735      0.814      0.626
Speed: 0.4ms preprocess, 23.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_shuffleattention_multihead
2024-05-03 10:23:24,471 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/Shuffle%20Attention%203%20May%202024%20Model%20Training%20Multi%20Head.ef6340197a6e45ceaf83fe6ffab81685/models/best.pt


Connecting multiple input models with the same name: `best`. This might result in the wrong model being used when executing remotely


## YOLOv8 + SPD-Conv

In [44]:
task.close()

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="SPD-Conv Retrain 22 May 2024 Model Training",
)


ClearML Task: created new task id=39b1b0489be04550813996fe2e49a1a0
2024-05-21 23:39:19,749 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/39b1b0489be04550813996fe2e49a1a0/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [6]:
# Step 2: Selecting the YOLOv8 Model, for this case SPD-Conv
model_variant = "ultralytics/cfg/models/v8/yolov8-SPD-Conv-New.yaml"
task.set_parameter("model_variant", model_variant)

In [7]:
# Step 3: Loading the YOLOv8 SPD-Conv Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-21 23:40:10,407 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 52/451 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): space_to_depth()
      (3): C2f(
        (cv1): Conv(
          (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03

In [8]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=32,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=300,
                      workers = 32,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_SPD_Conv',
                      save = True,
                      plots = True,
                      patience = 30,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': 32,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 300,
 'workers': 32,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_SPD_Conv',
 'save': True,
 'plots': True,
 'patience': 30,
 'verbose': True}

In [9]:
# Step 5: Initiating Model Training for SPD-Conv
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.18 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-SPD-Conv-New.yaml, data=/home/jupyter/gcs/una

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.28.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv7/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 73 weight(decay=0.0), 80 weight(decay=0.0005), 79 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 96 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv7
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.862      0.773      0.862      0.588

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    108/300      22.3G     0.9263     0.5889      1.284          1       1280: 100%|██████████| 37/37 [01:12<00:00,  1.96s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:14<00:00,  1.30it/s]


                   all        145        586      0.859      0.764      0.852      0.585
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 78, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

108 epochs completed in 2.642 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv7/weights/last.pt, 34.1MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv7/weights/best.pt, 34.1MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv7/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:14<00:00,  1.33it/s]


                   all        145        586      0.904      0.773      0.885      0.604
              backpack        145        192      0.907      0.758       0.88      0.568
               handbag        145        310      0.917      0.739      0.901      0.516
              suitcase        145         84      0.888      0.821      0.872      0.729
Speed: 0.8ms preprocess, 93.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_SPD_Conv7
2024-05-22 02:21:18,748 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/SPD-Conv%20Retrain%2022%20May%202024%20Model%20Training.39b1b0489be04550813996fe2e49a1a0/models/best.pt


## YOLOv8 + SPD-Conv + AIFI

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="SPD-Conv+AIFI 14 May 2024 Model Training",
)


ClearML Task: created new task id=ec090ed63a2a4e4cb253549649733f66
2024-05-14 02:50:15,347 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/ec090ed63a2a4e4cb253549649733f66/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [11]:
# Step 2: Selecting the YOLOv8 Model, for this case AIFI + SPD-Conv
model_variant = "ultralytics/cfg/models/v8/yolov8-aifi-SPD-Conv.yaml"
task.set_parameter("model_variant", model_variant)

In [12]:
# Step 3: Loading the YOLOv8 AIFI + SPD-Conv Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-14 02:56:08,412 - clearml.model - WARNING - Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely
Transferred 52/445 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): space_to_depth()
      (3): C2f(
        (cv1): Conv(
          (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03

In [13]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=-1,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=100,
                      workers = 8,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_AIFI_SPD_Conv',
                      save = True,
                      plots = True,
                      patience = 25,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': -1,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 100,
 'workers': 8,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_AIFI_SPD_Conv',
 'save': True,
 'plots': True,
 'patience': 25,
 'verbose': True}

In [14]:
# Step 5: Initiating Model Training for AIFI + SPD-Conv
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.15 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-aifi-SPD-Conv.yaml, data=/home/jupyter/gcs/un

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.27.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD_Conv2/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 72 weight(decay=0.0), 81 weight(decay=0.0005), 82 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 64 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD_Conv2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586      0.922      0.777      0.868      0.591

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      19.7G     0.6445     0.4154      1.139          2       1280: 100%|██████████| 73/73 [01:36<00:00,  1.32s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:17<00:00,  2.13it/s]


                   all        145        586      0.931      0.766      0.869       0.59

100 epochs completed in 3.198 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD_Conv2/weights/last.pt, 27.0MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD_Conv2/weights/best.pt, 27.0MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD_Conv2/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:17<00:00,  2.16it/s]


                   all        145        586      0.923      0.758      0.873      0.596
              backpack        145        192      0.933      0.724      0.866      0.556
               handbag        145        310      0.946      0.781      0.892       0.52
              suitcase        145         84       0.89       0.77       0.86       0.71
Speed: 0.6ms preprocess, 113.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_SPD_Conv2
2024-05-14 06:10:04,531 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/SPD-Conv%2BAIFI%2014%20May%202024%20Model%20Training.ec090ed63a2a4e4cb253549649733f66/models/best.pt


## YOLOV8 + GiraffeDet

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="Unattended Baggage Detection",
    task_name="GiraffeDet 14 May 2024 Model Training",
)


ClearML Task: created new task id=287d20eecd0b4e16a116bbe8700c01a1
2024-05-14 12:39:15,793 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/b0f06949b2114ff49e9e63675ea0f29b/experiments/287d20eecd0b4e16a116bbe8700c01a1/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [5]:
# Step 2: Selecting the YOLOv8 Model, for this case giraffeDet
model_variant = "ultralytics/cfg/models/v8/yolov8-giraffeDet.yaml"
task.set_parameter("model_variant", model_variant)

In [6]:
# Step 3: Loading the YOLOv8 giraffeDet Model
from ultralytics import YOLO
# Load a model
model = YOLO(f'{model_variant}')
model.load('yolov8n.pt') # loading pretrain weights

WARNING ⚠️ no model scale passed. Assuming scale='n'.
2024-05-14 12:40:20,948 - clearml.model - INFO - Selected model id: 784b431658954ef096968ba8b9c4dedd
Transferred 259/403 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [7]:
# Step 4: Setting Up Training Arguments
args = dict(data='/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
                      imgsz=1280,
                      batch=-1,
                      device = [0,1,2,3,4,5,6,7],
                      epochs=125,
                      workers = 8,
                      optimizer = 'SGD',
                      save_period = 5,
                      name = 'unattended_baggage_detection_AIFI_giraffeDet',
                      save = True,
                      plots = True,
                      patience = 30,
                      verbose = True)
task.connect(args)

{'data': '/home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/data.yaml',
 'imgsz': 1280,
 'batch': -1,
 'device': [0, 1, 2, 3, 4, 5, 6, 7],
 'epochs': 125,
 'workers': 8,
 'optimizer': 'SGD',
 'save_period': 5,
 'name': 'unattended_baggage_detection_AIFI_giraffeDet',
 'save': True,
 'plots': True,
 'patience': 30,
 'verbose': True}

In [9]:
# Step 5: Initiating Model Training for GiraffeDet
results = model.train(**args)

New https://pypi.org/project/ultralytics/8.2.15 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=ultralytics/cfg/models/v8/yolov8-giraffeDet.yaml, data=/home/jupyter/gcs/unatt

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************


Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            CUDA:4 (NVIDIA L4, 22491MiB)
                                                            CUDA:5 (NVIDIA L4, 22491MiB)
                                                            CUDA:6 (NVIDIA L4, 22491MiB)
                                                            CUDA:7 (NVIDIA L4, 22491MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
Overriding model.yaml nc=80 with nc=3
WARNING ⚠️ no model scale passed. Assuming scale='n'.
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/train/labels.cache... 1159 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1159/1159 [00:00<?, ?it/s]
val: Scanning /home/jupyter/gcs/unattended-baggage/unattended_bag_april_2024_experiment/valid/labels.cache... 145 images, 0 backgrounds, 0 corrupt: 100%|██████████| 145/145 [00:00<?, ?it/s]


Plotting labels to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_giraffeDet2/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 65 weight(decay=0.0), 72 weight(decay=0.0005), 71 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 64 dataloader workers
Logging results to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_giraffeDet2
Starting training for 125 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        145        586       0.88      0.767      0.863      0.574

125 epochs completed in 1.014 hours.
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_giraffeDet2/weights/last.pt, 6.3MB
Optimizer stripped from /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_giraffeDet2/weights/best.pt, 6.3MB

Validating /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_giraffeDet2/weights/best.pt...
Ultralytics YOLOv8.2.2 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (NVIDIA L4, 22491MiB)
                                                            CUDA:1 (NVIDIA L4, 22491MiB)
                                                            CUDA:2 (NVIDIA L4, 22491MiB)
                                                            CUDA:3 (NVIDIA L4, 22491MiB)
                                                            

Exception in thread Thread-2 (_pin_memory_loop):
Traceback (most recent call last):
  File "/opt/conda/envs/pytorch/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/pytorch/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 49, in _pin_memory_loop
    do_one_step()
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/pin_memory.py", line 26, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/envs/pytorch/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/multiprocessing/reductions.py", line 305, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/envs/pytorch/lib/python3.10/multiprocessing/resource_s

YOLOv8-giraffeDet summary: 232 layers, 2982937 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:03<00:00, 11.69it/s]


                   all        145        586       0.88      0.767      0.863      0.574
              backpack        145        192       0.87      0.802      0.846      0.523
               handbag        145        310      0.935      0.701      0.875      0.494
              suitcase        145         84      0.835      0.798      0.869      0.705
Speed: 0.6ms preprocess, 14.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /home/jupyter/unattended-baggage-yolov8-modification/runs/detect/unattended_baggage_detection_AIFI_giraffeDet2
2024-05-14 13:44:42,557 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Unattended%20Baggage%20Detection/GiraffeDet%2014%20May%202024%20Model%20Training.287d20eecd0b4e16a116bbe8700c01a1/models/best.pt
